In [1]:
import warnings
warnings.filterwarnings("ignore")

# Data Import 

In [2]:
import pandas as pd
import numpy as np

order_report = pd.read_excel(r'data/company_order_report.xlsx')
sku_master = pd.read_excel(r'data/company_sku_master.xlsx')
courier_invoice = pd.read_excel(r'data/courier_invoice.xlsx')
pincode_zones = pd.read_excel(r'data/company_pincode_zones.xlsx')
courier_rates = pd.read_excel(r'data/courier_rates.xlsx')
courier_rates["Zone"] = courier_rates["Zone"].str.lower()

# Exploring Data

#### Company Order Report

In [3]:
order_report.head(3)

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0


#### Company SKU Master

In [4]:
sku_master.head(3)

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113


#### Courier Company Invoice

In [5]:
courier_invoice.head(3)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.5,121003,532484,d,Forward charges,224.6


#### Company Pincode Zones

In [6]:
pincode_zones.head(3)

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d


#### Courier Company Rates

In [7]:
courier_rates.head(3)

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.25,29.5,23.6,13.6,23.6
1,b,0.50,33.0,28.3,20.5,28.3
2,c,0.75,40.1,38.9,31.9,38.9


# Calculating total weight as per company 'X'

In [8]:
# Merging tables order_report and sku
order_sku_merged = pd.merge(order_report, sku_master, on = 'SKU')
order_sku_merged["weight x qty"] = (order_sku_merged['Order Qty'] * order_sku_merged['Weight (g)'])/1000
# Grouping data w.r.t ExternOrderNo to find total weight of each ExternOrderNo
grouped_data = order_sku_merged.groupby('ExternOrderNo')['weight x qty'].sum().reset_index()
grouped_data.rename(columns = {"ExternOrderNo":"order_id", "weight x qty":'total wt by X'}, inplace = True)
grouped_data.head(5)

,order_id,total wt by X
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245


# Calculating total wt and zone by courier company and X

In [9]:
# Merging grouped_data and courier_invoice for AWB code, Charged Wt and Zone by courier company
merged1 = pd.merge(grouped_data, courier_invoice, left_on = "order_id", right_on = "Order ID")
merged1.head(3)

,order_id,total wt by X,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.22,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.48,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2


In [10]:
# Merging merged1 with pincode_zones for zone by X
merged2 = pd.merge(merged1, pincode_zones, how = "left", on = "Customer Pincode").drop_duplicates().reset_index()
merged2.head(3)

,index,order_id,total wt by X,AWB Code,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Warehouse Pincode_y,Zone_y
0,0,2001806210,0.22,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,121003,b
1,1,2001806226,0.48,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,121003,d
2,2,2001806229,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,121003,d


In [11]:
# Picking required columns
final = merged2[["order_id", "AWB Code", "total wt by X", "Charged Weight", "Zone_y", "Zone_x"]]
final.rename(columns = {"Charged Weight": "total wt by courier", "Zone_x": "zone by courier", "Zone_y":"zone by X"}, inplace = True)   
final.head(3)

,order_id,AWB Code,total wt by X,total wt by courier,zone by X,zone by courier
0,2001806210,1091117221940,0.22,2.92,b,b
1,2001806226,1091117222065,0.48,0.68,d,d
2,2001806229,1091117222080,0.50,0.71,d,d


# Weight Slab Calculator Function

In [12]:
def slab_calculator(table, zone, wt):
    wt_slab_a = 0.25
    wt_slab_b = 0.5
    wt_slab_c = 0.75
    wt_slab_d = 1.25
    wt_slab_e = 1.5
    slab = list()
    n_slab = list()
    
    for row in range(len(table)):
        if table[f"{zone}"][row] == "a":
            no_of_slabs = np.ceil(table[f"{wt}"][row]/wt_slab_a)
            total_wt_slab = wt_slab_a * no_of_slabs
            n_slab.append(no_of_slabs)
            slab.append(total_wt_slab)
            
        elif table[f"{zone}"][row] == "b":
            no_of_slabs = np.ceil(table[f"{wt}"][row]/wt_slab_b)
            total_wt_slab = wt_slab_b * no_of_slabs
            n_slab.append(no_of_slabs)
            slab.append(total_wt_slab)
            
        elif table[f"{zone}"][row] == "c":
            no_of_slabs = np.ceil(table[f"{wt}"][row]/wt_slab_c)
            total_wt_slab = wt_slab_c * no_of_slabs
            n_slab.append(no_of_slabs)
            slab.append(total_wt_slab)
            
        elif table[f"{zone}"][row] == "d":
            no_of_slabs = np.ceil(table[f"{wt}"][row]/wt_slab_d)
            total_wt_slab = wt_slab_d * no_of_slabs
            n_slab.append(no_of_slabs)
            slab.append(total_wt_slab)
            
        elif table[f"{zone}"][row] == "e":
            no_of_slabs = np.ceil(table[f"{wt}"][row]/wt_slab_e)
            total_wt_slab = wt_slab_e * no_of_slabs
            n_slab.append(no_of_slabs)
            slab.append(total_wt_slab)
            
    return slab, n_slab

# Wt Slab Calculation of Company X

In [13]:
final["wt slab by X"], final["n_slab_x"] = slab_calculator(final, "zone by X", "total wt by X")

# Wt Slab Calculation of Courier Company

In [14]:
final["wt slab by courier"], _ = slab_calculator(final, "zone by courier", "total wt by courier")

# Calculating Estimate Charge by X

In [15]:
# Merging courier bill and type of shipment
merge1 = pd.merge(final, courier_invoice[["Order ID", "Type of Shipment", "Billing Amount (Rs.)"]], left_on = "order_id", right_on = "Order ID")
merge1.drop("Order ID", axis = 1, inplace = True)
# Merging courier_rates for forward and rto rates
merge2 = pd.merge(merge1, courier_rates, left_on = "zone by X", right_on = "Zone").drop(["Zone", "Weight Slabs"], axis = 1)

In [16]:
# Calculating estimate charges by X
estimate_charge = list()

for row in range(len(merge2)):
    if merge2["Type of Shipment"][row] == "Forward charges":
        charge = ((merge2["n_slab_x"][row] - 1) * merge2["Forward Additional Weight Slab Charge"][row]) + merge2["Forward Fixed Charge"][row]
        estimate_charge.append(round(charge,1))
    elif merge2["Type of Shipment"][row] == "Forward and RTO charges":
        charge = (((merge2["n_slab_x"][row] - 1) * merge2["Forward Additional Weight Slab Charge"][row]) + merge2["Forward Fixed Charge"][row]) + (((merge2["n_slab_x"][row] - 1) * merge2["RTO Additional Weight Slab Charge"][row]) + merge2["RTO Fixed Charge"][row])          
        estimate_charge.append(round(charge,1))
        
merge2["Charge by X"]= estimate_charge

# Selecting Required Rows for final operations
df_final = merge2[[
    "order_id",
    "AWB Code", 
    "total wt by X",
    "wt slab by X",
    "total wt by courier",
    "wt slab by courier",
    "zone by X", 
    "zone by courier",
    "Charge by X",
    "Billing Amount (Rs.)"
]]

df_final.head(3)

,order_id,AWB Code,total wt by X,wt slab by X,total wt by courier,wt slab by courier,zone by X,zone by courier,Charge by X,Billing Amount (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.00,b,b,33.0,174.5
1,2001806233,1091117222135,0.245,0.5,0.78,1.00,b,b,33.0,61.3
2,2001806304,1091117222360,0.500,0.5,0.71,1.25,b,d,33.0,90.2


# Calculating Difference between charges by X and Courier Company

In [17]:
df_final["Difference Between Expected Charges and Billed Charges (Rs.)"] = df_final["Charge by X"] - df_final["Billing Amount (Rs.)"]
df_final.head(3)

,order_id,AWB Code,total wt by X,wt slab by X,total wt by courier,wt slab by courier,zone by X,zone by courier,Charge by X,Billing Amount (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.00,b,b,33.0,174.5,-141.5
1,2001806233,1091117222135,0.245,0.5,0.78,1.00,b,b,33.0,61.3,-28.3
2,2001806304,1091117222360,0.500,0.5,0.71,1.25,b,d,33.0,90.2,-57.2


# Renaming columns

In [18]:
df_final.rename(columns = {
    "order_id":"Order ID",
    "AWB Code":"AWB Number",
    "total wt by X":"Total weight as per X (KG)",
    "wt slab by X":"Weight slab as per X (KG)",
    "total wt by courier":"Total weight as per Courier Company (KG)",
    "wt slab by courier":"Weight slab charged by Courier Company (KG)",
    "zone by X":"Delivery Zone as per X",
    "zone by courier":"Delivery Zone charged by Courier Company",
    "Charge by X":"Expected Charge as per X (Rs.)",
    "Billing Amount (Rs.)":" Expected Charge as per X (Rs.)",
}, inplace = True)

In [19]:
# Saving file in excel format
df_final.to_excel(r"output/Output Data 1.xlsx", index = False)

In [20]:
df_final.head(3)

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.00,b,b,33.0,174.5,-141.5
1,2001806233,1091117222135,0.245,0.5,0.78,1.00,b,b,33.0,61.3,-28.3
2,2001806304,1091117222360,0.500,0.5,0.71,1.25,b,d,33.0,90.2,-57.2


# Summary Table 

In [21]:
# Importing only necessary columns for summary table
summary = df_final[["Difference Between Expected Charges and Billed Charges (Rs.)", "Expected Charge as per X (Rs.)"]]
summary.rename(columns = {
    "Difference Between Expected Charges and Billed Charges (Rs.)":"difference",
    "Expected Charge as per X (Rs.)":"amount"
}, inplace = True)
summary.head(3)

,difference,amount
0,-141.5,33.0
1,-28.3,33.0
2,-57.2,33.0


In [22]:
pos_amount = 0
pos_count = 0
neg_amount = 0
neg_count = 0
neu_amount = 0
neu_count = 0

for row in range(len(summary)):
    if summary["difference"][row] > 0:
        pos_amount += summary["difference"][row]
        pos_count += 1
        
    elif summary["difference"][row] < 0:
        neg_amount += summary["difference"][row]
        neg_count += 1
        
    elif summary["difference"][row] == 0:
        neu_amount += summary["amount"][row]
        neu_count += 1
        
print(pos_amount)
print(pos_count)
print(neg_amount)
print(neg_count)
print(neu_amount)
print(neu_count)

182.39999999999998
9
-6382.900000000005
101
881.2999999999998
14


In [23]:
# Creating DataFrame
df_summary = pd.DataFrame({
    None: ["Total orders where X has been correctly charged", "Total Orders where X has been overcharged", "Total Orders where X has been undercharged "],
    "Count":[neu_count, neg_count, pos_count],
    "Amount":[neu_amount, neg_amount, pos_amount]
             })

df_summary

,None,Count,Amount
0,Total orders where X has been correctly charged,14,881.3
1,Total Orders where X has been overcharged,101,-6382.9
2,Total Orders where X has been undercharged,9,182.4


In [24]:
# Exporting Data
df_summary.to_excel("output/Output Data 2.xlsx", index = False)